In this program I've built a bayesian classifier for cancer types:'GBM','LAML','LGG','PAAD','PRAD'. The features selected for classification were gene expression values (via TCGA) of 11 genes chosen for their role in immune function or known association with certain cancer types. The goal was to determine whether a bayesian classifier could classify a cancer type based on the expression values of these 11 genes. 

Preliminary results with this classifer show a high accuracy (90-94%) of cancer type prediction depending on number of principle components used to train classifier. The cancer dataset consisted of the gene expression values of 11 genes from 1532 patients across 5 cancer types ('GBM','LAML','LGG','PAAD','PRAD' ).   

In [39]:
import os, struct
import matplotlib as plt
import numpy as np
import numpy.linalg as LA
import pandas as pd
from pylab import *
import random
import operator
from array import array as pyarray

def readExcelSheet1(excelfile):
    from pandas import read_excel
    return (read_excel(excelfile)).values

#This function is used in the function readExcel(...) defined further below
def readExcelRange(excelfile,sheetname="Sheet1",startrow=1,endrow=1,startcol=1,endcol=1):
    from pandas import read_excel
    values=(read_excel(excelfile, sheetname,header=None)).values;
    return values[startrow-1:endrow,startcol-1:endcol]

#This is the function you can actually use within your program.
#See manner of usage further below in the section "Prepare Data"

def readExcel(excelfile,**args):
    if args:
        data=readExcelRange(excelfile,**args)
    else:
        data=np.array(readExcelSheet1(excelfile))
    if data.shape==(1,1):
        return data[0,0]
    elif (data.shape)[0]==1:
        return data[0]
    else:
        return data

def writeExcelData(x,excelfile,sheetname,startrow,startcol):
    from pandas import DataFrame, ExcelWriter
    from openpyxl import load_workbook
    df=DataFrame(x)
    book = load_workbook(excelfile)
    writer = ExcelWriter(excelfile, engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    df.to_excel(writer, sheet_name=sheetname,startrow=startrow-1, startcol=startcol-1, header=False, index=False)
    writer.save()
    writer.close()

def getSheetNames(excelfile):
    from pandas import ExcelFile
    return (ExcelFile(excelfile)).sheet_names
sheetname = 'Results'
startcol = 2
excelfile=r"/Volumes/Macintosh HD/Users/louisecabansay/Dropbox (Personal)/CancerHackers/CancerGeneData.xlsx";



## Creating Randomized Test/Train Sets

In [40]:
def TestTrainDataSplit(dataset, split):#dataset = full dataset; 
    #split = percent of dataset to be training set (enter as decimal)
    trainingSet = []
    testSet = []
    for x in range(len(dataset)):
            dataset[x] = dataset[x]
            if random.random() < split:
                trainingSet.append(dataset[x])
            else:
                testSet.append(dataset[x])
    return np.array(trainingSet), np.array(testSet)


## PCA Functions

In [41]:
def XUZCVPR(dataset, Utrain, train):
    #this function runs a principle components analysis on the data set where X is the data matrix without class labels
    X = np.array(dataset)
    #compute mean vector, U
    if train==True:
        Uvector = np.mean(X,axis=0)
        U = np.array([Uvector])
    else: 
        U=Utrain
    #compute mean difference vector Z
    Z = X - U 
    meanZ = np.mean(Z,axis=0) # axis to calculate column means, should be 0
    meanZround = [round(x) for x in meanZ]
    emptymeanZ=filter(lambda x:x != 0, meanZround) # all the column mean of z should be 0
    
    #covariance matrix, C
    C = np.cov(Z.astype(float),rowvar=False)
    Ctranspose = C.transpose()
    checkC = np.array_equal(C,Ctranspose) #make sure covariance matrix has NxN dimensions
    
    aEighV=LA.eigh(C)#eigen values in decending order
    V = flipud(aEighV[1].T)
    Evals = flipud(aEighV[0])#Evals = eigen values
    Vrows = V[0,:]
    checkVrows = (np.dot(C, Vrows))/(Evals[0]*Vrows)#verifies that the rows (not columns) of matrix are eigen vectors
    
    P=np.dot(Z,V.T) #Principle components projection matrix
    R=np.dot(P,V)
    Xrec = R+U #recover X (using all principle components) to verify successful PCA
    #once Xrec has been verified for all components, use DimensionReduction() to see the difference between X and Xrec
    #using differing princple components (ex: top 2, 5, 10etc)
    
    print 'X-shape: ' +repr(X.shape)
    print 'U-shape: ' +repr(U.shape)
    print 'Z-shape: ' +repr(Z.shape)
    print 'C-shape: ' +repr(C.shape)
    print 'V-shape: ' +repr(V.shape)
    print 'P-shape: ' +repr(P.shape)
    print 'R-shape: ' +repr(R.shape)
    print 'Xrec-shape: '+ repr(Xrec.shape)
    print 'meanZround: ' + repr(meanZround)
    print 'emptymeanZ: ' + repr(emptymeanZ)
    print 'C equals C.T : ' + repr(checkC)
    print 'Rows are eigenvectors if values are 1: ' + repr(checkVrows)    
    print 'Note: Eigenvectors and values returned in order most to least importance'
    return np.array(X), np.array(U), np.array(Z), np.array(C), np.array(V), np.array(Evals), np.array(P), np.array(R), np.array(Xrec)
#[X,U,Z, C, V, Evals, P, R, Xrec]

def DimensionReduction(X, P, V, U):
    reducedDims = []
    Xdiffavg = []
    for d in range(len(U.T)):
        i = d+1
        Xrec = (np.dot(P[:,0:i],V[0:i,:]))+U
        reducedDims.append(np.array(Xrec))

    for m in range(len(U.T)):
        Xdiffnorms = []
        for w in range(len(X)):
            tXdim = reducedDims[m][w]
            Xdiffs = X[w]-tXdim
            normXdiff = LA.norm(Xdiffs)
            Xdiffnorms.append(normXdiff)
        meanXdiff = np.mean(Xdiffnorms)
        Xdiffavg.append(meanXdiff)
    return np.array(Xdiffavg)

def printPCAresults(results, numPC):
    for a in range(numPC):
        print 'Using '+repr(a+1)+' principle component(s) the average difference between X and Xrec is '+repr(results[a]) 

    

## Bayesian Classifier Functions

In [42]:
def BuildNDBayesianClassifier(Dataset, Classlabels, D):
    ClassStats = {}
    for n in range(len(Classlabels)):
        ClassStats[Classlabels[n]]={}
        Class = Dataset[Dataset[:,-1] == Classlabels[n]]
        ClassData = (Class[:,:D]).astype(float)
        ClassStats[Classlabels[n]]['Num'] = len(Class)
        ClassStats[Classlabels[n]]['Data'] = ClassData
        ClassStats[Classlabels[n]]['Mean'] = np.mean(ClassData,axis=0)
        ClassStats[Classlabels[n]]['Cov'] = np.cov(ClassData, rowvar=False)
    return ClassStats
    

def ApplyNDBayesianClassifier(TestDataset, TrainDataset, Classlabels, D):
    ClassStats = BuildNDBayesianClassifier(TrainDataset, Classlabels, D)
    w=1; #width of the bin
    CountC_all = []
    for n in range(len(Classlabels)):
        NC = ClassStats[Classlabels[n]]['Num']
        UC = ClassStats[Classlabels[n]]['Mean']
        covC = ClassStats[Classlabels[n]]['Cov']
        testset = np.array((TestDataset[:,:D])).astype(float)
        countC = NC*w*pdf(testset, UC, covC)
        #print countC
        CountC_all.append(countC)
    [resultlabel, resultprob]= ResultLPBayesClassifier(CountC_all, TestDataset, Classlabels)
    return np.array([resultlabel, resultprob])

def ResultLPBayesClassifier(CountC_all, TestDataset, Classlabels):
    ClassCounts_all = np.array(CountC_all)
    print ClassCounts_all.shape
    resultlabel = np.full(np.alen(TestDataset), "Indeterminate", dtype=object)
    resultprob = np.full(np.alen(TestDataset), 0 , dtype=float)
    for g in range(len(TestDataset)):
        CountXvalues = []
        for w in range(len(Classlabels)):
            count = ClassCounts_all[w][g]
            CountXvalues.append(count)
        max_value = max(CountXvalues)
        max_index = CountXvalues.index(max_value)
        label = Classlabels[max_index]
        resultlabel[g]=label
        #print sum(ClassCounts_all)
        resultprob = (ClassCounts_all[max_index][g]).astype('float')/sum(ClassCounts_all)
    return resultlabel, resultprob
        

In [43]:
def pdf(x,mu,sigma):
    #print x
    #print mu
    xf = x.astype(float)
    muf = mu.astype(float)

    d=np.alen(muf)
    dfact1=(2*np.pi)**d
    dfact2=LA.det(sigma)
    fact=1/np.sqrt(dfact1*dfact2)
    xc=xf-muf
    isigma=LA.inv(sigma)
#    isigxc = np.dot(isigma,xc.T)
#    ex = np.dot(xc,isigxc)
    npdf = fact * np.exp(-0.5 * np.einsum('ij,jk,ik->i',xc,isigma,xc))
    return npdf  

## Performance Metrics: Functions to Evaluate and Print Bayes Classifier Results

In [44]:
def PerformanceMetrics(Resultlabels, Dataset, PositiveLabel, Classlabels):
    OutputCL = (Resultlabels).astype('str')
    GroundTruth = (Dataset[:,-1]).astype('str')
    Classcomps = OutputCL == GroundTruth
    #print Classcomps    
    TrueP=0
    FalseP=0
    TrueN=0
    FalseN=0
    Num=0
    for i in range(len(GroundTruth)):
        if Classcomps[i]== True:
            if OutputCL[i] == PositiveLabel:
                TrueP+=1
            else:
                TrueN+=1
        elif Classcomps[i]==False:
            if OutputCL[i] != PositiveLabel:
                FalseN+=1
            else:
                FalseP+=1
    Accuracy = float((TrueP+TrueN))/(TrueP+TrueN+FalseP+FalseN)
    Sensitivity = float((TrueP))/(TrueP+FalseN)
    Specificity = float((TrueN))/(FalseP+TrueN)
    if (TrueP+FalseP)==0:
        PPV = 'No Positive Predictions'
    else: 
        PPV = float((TrueP))/(TrueP+FalseP)
    stringmeasures = ['TrueP', 'FalseP', 'TrueN', 'FalseN', 'Accuracy','Sensitivity', 'Specificity','PPV']
    measures_values = [TrueP, FalseP, TrueN, FalseN, Accuracy, Sensitivity, Specificity, PPV]
    print 'Classifier Performance:'
    print '     Positive Class Label: '+ repr(PositiveLabel)
    for i in range(len(stringmeasures)):
        print '         '+stringmeasures[i]+ ': '+repr(measures_values[i])
    
    return [TrueP, FalseP, TrueN, FalseN, Accuracy, Sensitivity, Specificity, PPV]


def BayesPCAperformance(TestDataset, TrainDataset, Classlabels, dimensions, allD):
    BayesPCAperformance =[]
    DResults=EvaluateBayesPCA(TestDataset, TrainDataset, Classlabels, dimensions, allD)
    PC=len(DResults)
    if allD==False:
        PC = 1
    for i in range(PC):
        if allD==False:
            print repr(dimensions)+' Principal Components Bayes Classifier Performance:'
        else:
            print repr(i+2)+' Principal Components Bayes Classifier Performance:'
            
        labelstring = []
        reallabelstring = []
        OutputCL = (DResults[i][0]).astype('str')
        realCL=(TrainDataset[:,-1]).astype('str')
        for j in range(len(Classlabels)):
            labelnum = OutputCL[OutputCL == Classlabels[j]]
            lstring = repr(Classlabels[j])+ ': '+ repr(len(labelnum))
            labelstring.append(lstring)
            reallabelnum = realCL[realCL == Classlabels[j]]
            reallstring = repr(Classlabels[j])+ ': '+ repr(len(reallabelnum))
            reallabelstring.append(reallstring)
        print '  Training Output ->' + repr(reallabelstring) + ' Total: ' +repr(len(realCL))
        print '  Testing Output -->' + repr(labelstring)+ ' Total: ' +repr(len(OutputCL))
        totalPPV = []
        for cl in range(len(Classlabels)):
            PositiveLabel=Classlabels[cl]
            if allD==True:
                nDBayesPerformance = PerformanceMetrics(DResults[i][0], TestDataset, PositiveLabel, Classlabels)
                totalPPV.append(nDBayesPerformance[-1])
            else:
                nDBayesPerformance = PerformanceMetrics(DResults[0], TestDataset, PositiveLabel, Classlabels)
                totalPPV.append(nDBayesPerformance[-1])
            BayesPCAperformance.append(nDBayesPerformance)
        
        print 'Avg PPV: '+ repr(np.mean(totalPPV))   
        print '\n'
    return np.array(BayesPCAperformance)

def EvaluateBayesPCA(TestDataset, TrainDataset, Classlabels, dimensions, allD):
    DBayesPCAResults=[]
    if allD == True: 
        for d in range(dimensions-1):
            D=d+2
            nBResults = ApplyNDBayesianClassifier(TestDataset, TrainDataset, Classlabels, D)
            DBayesPCAResults.append(nBResults)
        return np.array(DBayesPCAResults)
    else:
        D=dimensions
        nBResults = ApplyNDBayesianClassifier(TestDataset, TrainDataset, Classlabels, D)
        return np.array(nBResults)
        
        

## Main Function: Load Data, Run PCA, Run Classifier

In [57]:
cdata=np.array(readExcel(excelfile))
print cdata.shape
#print cdata[:5,:]
random.seed(8)
[CTrain, CTest]=TestTrainDataSplit(cdata, 0.66)
print CTrain
print CTest.shape

(1532, 12)
[[630.0 44776.0 64147.0 ..., 6080.0 5599.0 u'GBM']
 [792.0 62668.0 3048.0 ..., 2575.0 1633.0 u'GBM']
 [1114.0 77348.0 11210.0 ..., 3055.0 2302.0 u'GBM']
 ..., 
 [372.6402 29000.0 7155.9508 ..., 1047.3324 1806.5663 u'PRAD']
 [308.6948 20300.5498 3498.2692 ..., 1344.7363 1768.6825 u'PRAD']
 [219.1653 23697.2099 6139.7373 ..., 1462.0347 1892.2826 u'PRAD']]
(512, 12)


In [46]:
[X10,U10,Z10, C10, V10, Evals10, P10, R10, Xrec10] = XUZCVPR(CTrain[:,:-1], Utrain=1, train=True)

X-shape: (1020, 11)
U-shape: (1, 11)
Z-shape: (1020, 11)
C-shape: (11, 11)
V-shape: (11, 11)
P-shape: (1020, 11)
R-shape: (1020, 11)
Xrec-shape: (1020, 11)
meanZround: [-0.0, 0.0, 0.0, -0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, -0.0]
emptymeanZ: []
C equals C.T : True
Rows are eigenvectors if values are 1: array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])
Note: Eigenvectors and values returned in order most to least importance


In [56]:
#Run PCA and view results for X and Xrec depending on number of principle components used
#WARNING: this function can be memory intensive depending on size of dataset
C10resultsPCA= DimensionReduction(X10, P10, V10, U10)
printPCAresults(C10resultsPCA, numPC=11) #check data set for how large X values are, if avg diff makes sense
#smallest X-Xrec difference will give best classification results.
#dimension reduction might not be possible in some datasets 

Using 1 principle component(s) the average difference between X and Xrec is 7257.2388186280914
Using 2 principle component(s) the average difference between X and Xrec is 3632.5150552861355
Using 3 principle component(s) the average difference between X and Xrec is 2519.5969502507114
Using 4 principle component(s) the average difference between X and Xrec is 1822.3653849135153
Using 5 principle component(s) the average difference between X and Xrec is 1373.7659988263226
Using 6 principle component(s) the average difference between X and Xrec is 1014.6559452189041
Using 7 principle component(s) the average difference between X and Xrec is 755.40956359763697
Using 8 principle component(s) the average difference between X and Xrec is 463.5205785975557
Using 9 principle component(s) the average difference between X and Xrec is 256.22913484231174
Using 10 principle component(s) the average difference between X and Xrec is 156.05535826736519
Using 11 principle component(s) the average differ

In [48]:
trainClabels = (CTrain[:,-1][...][None]).astype(str)#training set labels
testClabels = (CTest[:,-1][...][None]).astype(str) #test set labels
clabels= np.array(['GBM','LAML','LGG','PAAD','PRAD']) #classes by cancer gene tcga name 
trainClabels.shape

(1, 1020)

In [49]:
testCZ=(CTest[:,:-1])-U10
#subtract meanvector from testvals
V10.shape
testC5 = dot(testCZ,V10.T)
#get p vector for test
print testC5.shape

(512, 11)


In [50]:
trainOnco = np.concatenate((P10, trainClabels.T), axis=1)
print trainOnco.shape
#print trainOnco[:10,:]

(1020, 12)


In [51]:
testOnco = np.concatenate((testC5, testClabels.T), axis=1)
print testOnco.shape
#print testOnco[:10,:]

(512, 12)


In [52]:
cdimensions = 8
allD10 = False #whether or not to show classifier results from all PCA components up to cdimensions or just in cdimensions
MN10BayesPCAperformance = BayesPCAperformance(testOnco, trainOnco, clabels, cdimensions, allD10)

(5, 512)
8 Principal Components Bayes Classifier Performance:
  Training Output ->["'GBM': 107", "'LAML': 102", "'LGG': 346", "'PAAD': 137", "'PRAD': 328"] Total: 1020
  Testing Output -->["'GBM': 1", "'LAML': 1", "'LGG': 1", "'PAAD': 1", "'PRAD': 1"] Total: 3
Classifier Performance:
     Positive Class Label: 'GBM'
         TrueP: 35
         FalseP: 12
         TrueN: 435
         FalseN: 30
         Accuracy: 0.91796875
         Sensitivity: 0.5384615384615384
         Specificity: 0.9731543624161074
         PPV: 0.7446808510638298
Classifier Performance:
     Positive Class Label: 'LAML'
         TrueP: 67
         FalseP: 0
         TrueN: 403
         FalseN: 42
         Accuracy: 0.91796875
         Sensitivity: 0.6146788990825688
         Specificity: 1.0
         PPV: 1.0
Classifier Performance:
     Positive Class Label: 'LGG'
         TrueP: 171
         FalseP: 13
         TrueN: 299
         FalseN: 29
         Accuracy: 0.91796875
         Sensitivity: 0.855
         Spe

In [53]:
cdimensions = 8
allD10 = True #whether or not to show classifier results from all PCA components up to cdimensions or just in cdimensions
MN10BayesPCAperformance = BayesPCAperformance(testOnco, trainOnco, clabels, cdimensions, allD10)

(5, 512)
(5, 512)
(5, 512)
(5, 512)
(5, 512)
(5, 512)
(5, 512)
2 Principal Components Bayes Classifier Performance:
  Training Output ->["'GBM': 107", "'LAML': 102", "'LGG': 346", "'PAAD': 137", "'PRAD': 328"] Total: 1020
  Testing Output -->["'GBM': 29", "'LAML': 86", "'LGG': 209", "'PAAD': 22", "'PRAD': 166"] Total: 512
Classifier Performance:
     Positive Class Label: 'GBM'
         TrueP: 25
         FalseP: 4
         TrueN: 368
         FalseN: 115
         Accuracy: 0.767578125
         Sensitivity: 0.17857142857142858
         Specificity: 0.989247311827957
         PPV: 0.8620689655172413
Classifier Performance:
     Positive Class Label: 'LAML'
         TrueP: 69
         FalseP: 17
         TrueN: 324
         FalseN: 102
         Accuracy: 0.767578125
         Sensitivity: 0.40350877192982454
         Specificity: 0.9501466275659824
         PPV: 0.8023255813953488
Classifier Performance:
     Positive Class Label: 'LGG'
         TrueP: 157
         FalseP: 52
         Tru